In [ ]:
# Ultralytics YOLO model
from ultralytics import YOLO 

# Image reading 
import cv2

# OS traversal 
import os 

# Ploting 
import matplotlib.pyplot as plt

# Polygons 
from shapely.geometry import Polygon

# Array math 
import numpy as np

In [ ]:
# Defining the model to use 
path_to_model = os.path.join('..', 'runs', 'segment', 'train11', 'weights', 'last.pt') 

# Loading the model
model = YOLO(path_to_model)

In [ ]:
# Defining the path to the image 
path_to_image = os.path.join('input', 'DJI_0972-2023-11-30-11-21-12.jpg')

# Reading the image
img = cv2.imread(path_to_image)

# Going to RGB 
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# Ploting the original image
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
# Applying the model 
results = model(path_to_image)

# Extracting all the masks 
masks = results[0].masks.xy

In [ ]:
# Iterating over the masks and drawing a polygon 
for mask in masks:
    # Iterating over the mask and creating a polygon
    polygon_points = mask
    polygon_points = polygon_points.reshape((-1, 1, 2))

    # Converting to int
    polygon_points = polygon_points.astype(int)

    cv2.polylines(img, [polygon_points], isClosed=True, color=(255, 100, 100), thickness=5)

# Ploting the image with the polygons
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
def calculate_centroid(polygon):
    """ Calculate the centroid of a polygon. """
    x_coords = [p[0] for p in polygon]
    y_coords = [p[1] for p in polygon]
    centroid_x = sum(x_coords) / len(polygon)
    centroid_y = sum(y_coords) / len(polygon)
    return (centroid_x, centroid_y)

def calculate_distance(point1, point2):
    """ Calculate Euclidean distance between two points. """
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


In [ ]:
# Calculating the center point of the image 
center_point = (img.shape[1] / 2, img.shape[0] / 2)

In [ ]:
# Calculating the centroids of the masks 
centroids = []
for mask in masks:
    centroid = calculate_centroid(mask)
    centroids.append(centroid)

# Getting the closest mask
closest_mask = None
closest_distance = None
for mask, centroid in zip(masks, centroids):
    distance = calculate_distance(center_point, centroid)
    if closest_distance is None or distance < closest_distance:
        closest_distance = distance
        closest_mask = mask

In [ ]:
# Drawing the closest centroid with a blue line
closest_mask = closest_mask.reshape((-1, 1, 2))
closest_mask = closest_mask.astype(int)
cv2.polylines(img, [closest_mask], isClosed=True, color=(100, 100, 255), thickness=5)

# Ploting the image with the polygons
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
# Get width and height coordinates from image center
w, h = center_point

# Create bbox around image center
center_box = [w - 150, h - 150, w + 150, h + 150]
center_box = [int(x) for x in center_box]

# Draw bbox on image
img = cv2.rectangle(
    img,
    (center_box[0], center_box[1]),
    (center_box[2], center_box[3]),
    (255, 0, 0),
    2,
)

# Ploting the image with the center bbox
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
# Get width and height coordinates from image center
w, h = center_point
# Create bbox around image center
center_box = [w - 150, h - 150, w + 150, h+ 150]
center_box = [int(x) for x in center_box]

In [ ]:
img = cv2.rectangle(
    img,
    (center_box[0], center_box[1]),
    (center_box[2], center_box[3]),
    (255, 0, 0),
    2,
)

In [ ]:
# Convert bounding box into polygon
center_bbox_shape = Polygon(
    [
        (center_box[0], center_box[1]),
        (center_box[2], center_box[1]),
        (center_box[2], center_box[3]),
        (center_box[0], center_box[3]),
    ]
)

In [ ]:
# Convert numpy array to list
closest_mask_list = closest_mask.tolist()
# Wrangling list for polygon coversion
line_struct = [x[0] for x in closest_mask_list]
# Create closest mask polygon
segment = Polygon(line_struct)

In [ ]:
# Get intersected polygon boundries between bbox and closest mask polygons
intersection = segment.intersection(center_bbox_shape)

In [ ]:
# Get x, y coords from intersected polygon
x, y = intersection.exterior.coords.xy

# Create list of tuples with coordinates
coord_pairs = list(zip(x, y))

# Convert each tuple to list element
coord_pairs = [list([int(y) for y in x]) for x in coord_pairs]

# Convert list to numpy array
intersection_poly = np.array(coord_pairs)

In [ ]:
# Draw poly on image
cv2.polylines(
    img, [intersection_poly], isClosed=False, color=(0, 255, 255), thickness=5
)

In [ ]:
# Ploting the image with the polygons
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
# Converting the image to grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Create a mask
mask = np.zeros(img.shape[:2], dtype=np.uint8)

# Set mean value to 1 if no intersection between center bbox
# and closest mask was found
if intersection.area == 0:
    mean_val = 1
else:
    # Fill the polygon on the mask
    cv2.fillPoly(mask, [intersection_poly], 255)

    # Apply the mask to the image
    masked_image = cv2.bitwise_and(img_gray, img_gray, mask=mask)

    # Calculate the mean pixel value
    # Use mask to ignore zero pixels in the mean calculation

    mean_val = cv2.mean(masked_image, mask=mask)

    # Limiting the mean value to 0 - 1
    mean_val = np.clip(mean_val[0] / 255, 0, 1)

    # Rounding to 2 decimals
    mean_val = round(mean_val, 2)

# Ploting the image with the polygons
plt.figure(figsize=(10, 10))
plt.imshow(img)

# Adding the mean value to the image
plt.title(f"Mean value: {mean_val}", fontsize=15, color="red")